In [2]:
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

In [ ]:
# torch = { version = "=1.12.1+cu113", source = "torch" }

# [[tool.poetry.source]]
# name = "torch"
# url = "https://download.pytorch.org/whl/cu113"
# secondary = true
# default = false

In [3]:
# df = pd.read_parquet('C:/Users/sgopalakrish/Downloads/intellizenz-model-training/data/export_features_2016_2020_v3.parquet.gzip')
df = pd.read_parquet('/Users/saravana/Documents/Work/Master-Thesis/reclamation-processing/data/export_anonymized_features_2016_2020.parquet.gzip')

pd.set_option('display.max_columns', None)
display(df.head())

,anonymized_band,anonymized_promoter,anonymized_venue,tarif_bez,vg_state,vg_datum_season,vg_datum_month,vg_datum_day_of_week,vg_datum_year,vg_datum_von,veranst_segment,vg_inkasso
ID,,,,,,,,,,,,
9938766,Klapp Dehmel AG & Co. OHG,Wiek Löchel AG,turnhalle,U-K (MUSIKER),BAVARIA,0,1,5,0,2014-02-08,2,210.00
9939272,Karz Ritter KGaA,Peukert Neuschäfer GmbH,grundschule,U-V II. 1 (+II 2),THURINGIA,0,1,5,0,2014-02-22,1,85.06
9939279,Klemm Ullrich OHG mbH,Naser Klemt KG,werk,U-V II. 1 (+II 2),BAVARIA,0,1,1,0,2014-02-18,1,90.20
9939280,Albers Klemt GmbH & Co. OHG,Patberg AG & Co. KG,kulturhaus,U-V II. 1 (+II 2),BAVARIA,1,2,5,0,2014-03-29,2,206.12
9939281,Albers Klemt GmbH & Co. OHG,Patberg AG & Co. KG,kulturhaus,U-V II. 1 (+II 2),BAVARIA,1,2,4,0,2014-03-21,2,206.12


In [42]:
df1 = pd.read_parquet('/Users/saravana/Documents/Work/Master-Thesis/reclamation-processing/data/export_anonymized_features_2016_2020_v1.parquet.gzip'
)
pd.set_option('display.max_columns', None)
display(df1.head())

,anonymized_band,anonymized_promoter,anonymized_venue,tarif_bez,vg_state,vg_datum_season,vg_datum_month,vg_datum_day_of_week,vg_datum_year,vg_datum_von,veranst_segment,vg_inkasso,place_kirche,place_hotel,place_cafe,place_theater,place_club,place_halle,place_kulturzentrum,place_gaststaette,place_buergerhaus,place_festhalle,place_stadthalle,place_festzelt,place_schloss,place_pub,place_restaurant,place_gasthaus,place_bar,place_kurhaus,place_kulturhaus,place_kabarett,place_rathaus,place_arena,place_gasthof,place_park,place_wandelhalle,place_schlachthof,place_turnhalle,place_staatsbad,place_zelt,place_mehrzweckhalle,tarif_u-v ii. 1 (+ii 2),tarif_u-k (musiker),tarif_u-k (musiker) mindestverguetung,tarif_u-st i. (musiker),tarif_u-k i (+ ii 2b) - mit sonstigem geldwerten vorteil,tarif_u-st i (musiker) nl,tarif_u-v iii. 1,tarif_u-v iii. 2,tarif_u-k iii. 2d) (musiker) - vor geladenen gaesten,tarif_variete i (musiker),tarif_u-v vi. b,tarif_e (musiker),tarif_p-k i. (u-musik),tarif_u-k ii. (musiker) bis 50 min,tarif_u-k ii. (musiker) bis 20 min,tarif_e-p,tarif_u-k ii. (musiker) bis 25 min,tarif_u-k ii. (musiker) bis 15 min,tarif_u-k ii. (musiker) bis 30 min,tarif_u-k ii. (musiker) bis 10 min,tarif_u-st (musiker),tarif_chorverband konzert u-musik,tarif_vk i 3 zirkusunternehmen (musiker),tarif_u-k ii. (musiker) bis 5 min,tarif_u-k ii. (musiker) bis 35 min,tarif_u-k ii. (musiker) bis 40 min,tarif_p-k i. (e-musik),tarif_u-k ii (musiker) mindestverguetung,tarif_u-k ii. (musiker) bis 45 min,tarif_u-v vi. b mindestverguetung,state_bavaria,state_baden-wuerttemberg,state_north rhine-westphalia,state_hesse,state_lower saxony,state_berlin,state_rhineland-palatinate,state_saxony,state_hamburg,state_thuringia,state_schleswig-holstein,state_brandenburg,state_saxony-anhalt,state_mecklenburg-western pomerania,state_saarland,state_bremen,band_hofmann ortmann ag,band_christoph löffler ag,band_margraf döring gbr,band_huhn trüb ag & co. ohg,band_finke haering ag,band_pergande neuschäfer ag,band_hornig täsche ohg mbh,band_ziegert scheel ag,band_adolph heser gmbh & co. ohg,band_hübel eckbauer ag,band_kensy gute gmbh & co. kgaa,band_ring flantz stiftung & co. kg,band_wende scheuermann gmbh & co. kg,band_pärtzelt bolnbach kg,band_bachmann ernst kg,band_hamann hethur stiftung & co. kgaa,band_nohlmans roskoth e.v.,band_schönland koch ii ag,band_hermighausen süßebier gmbh,band_juncken briemer stiftung & co. kg,band_lachmann heinrich gmbh & co. ohg,band_hendriks döring ag,band_bähr weihmann ag,band_beer reising ag,band_stumpf ritter gmbh & co. kgaa,band_thies stiebitz gbr,band_bärer jungfer gmbh & co. kg,band_trubin butte e.g.,band_römer steckel gmbh & co. kg,band_wiek schmiedt gmbh & co. ohg,promoter_schmidtke reichmann gmbh & co. kg,promoter_heintze ag & co. kg,promoter_drubin gmbh,promoter_textor kg,promoter_schleich stiftung & co. kg,promoter_fiebig weinhage ag & co. kgaa,promoter_zobel carsten gbr,promoter_finke hethur e.g.,promoter_fröhlich sölzer gmbh,promoter_seip scholtz gmbh,promoter_margraf dörr ag & co. kg,promoter_bolzmann caspar gmbh & co. ohg,promoter_franke franke ag,promoter_sontag gmbh & co. ohg,promoter_wesack riehl gmbh & co. kg,promoter_röhrdanz,promoter_juncken eckbauer e.v.,promoter_gutknecht fechner ag,promoter_knappe ag,promoter_flantz kensy gmbh & co. ohg,promoter_kohl kabus ag & co. kg,promoter_gunpf hertrampf ohg mbh,promoter_förster van der dussen ag & co. kg,promoter_mude eckbauer gmbh,promoter_metz ohg mbh,promoter_franke langern stiftung & co. kg,promoter_biggen hermann ag & co. ohg,promoter_wähner haase gmbh & co. kg,promoter_dippel koch kg,promoter_kitzmann ag
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
9938766,Klapp Dehmel AG & Co. OHG,Wiek Löchel AG,turnhalle,U-K (MUSIKER),BAVARIA,0,1,5,0,2014-02-08,2,210.00,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [43]:
df1.shape

(1742197, 148)

In [4]:
df.shape

(1742197, 12)

In [5]:
df['tarif_bez'].unique()

['U-K (MUSIKER)', 'U-V II. 1 (+II 2)', 'U-K (MUSIKER) MINDESTVERGUETUNG', 'U-K I (+ II 2B) - MIT SONSTIGEM GELDWERTEN VO..., 'U-K II. (MUSIKER) BIS 50 MIN', ..., 'WR-HZ (MUSIKER)', 'KONZERT CHORVERBAENDE OHNE EINTRITTSGELD (7,5%)', 'Z I. 1./3. (MUSIKER)', 'BM I. 1. / 2.', 'RV/L NR. 1 (3) I 1. PROZENTABRECHNUNG TONTRAE...]
Length: 87
Categories (91, object): ['BM I. 1. / 2.', 'BM I.1. BIS 25 MINUTEN', 'CHORVERBAND KONZERT E-MUSIK MIT EINTRITTSGELD', 'CHORVERBAND KONZERT E-MUSIK OHNE EINTRITTSGELD', ..., 'WR-KS I. 2 EINZEL (MUSIKER)', 'WR-NWSP I.', 'WR-VR-B 1 I (MUSIKER)', 'Z I. 1./3. (MUSIKER)']

In [44]:
print(len(df1['tarif_bez'].unique()))

87


### Get all unique values of Tarif that contains 'U-ST'

In [6]:
tarif_u_st_df = df.loc[df['tarif_bez'].str.contains('U-ST ')].copy()

In [7]:
display(tarif_u_st_df.head())

,anonymized_band,anonymized_promoter,anonymized_venue,tarif_bez,vg_state,vg_datum_season,vg_datum_month,vg_datum_day_of_week,vg_datum_year,vg_datum_von,veranst_segment,vg_inkasso
ID,,,,,,,,,,,,
9943647,Lübs Textor GmbH & Co. KG,Buchholz Langern AG,Juliane-Wiek-Gasse,U-ST (MUSIKER),SAXONY-ANHALT,0,0,5,0,2014-01-18,1,55.44
9946796,Hein Dörr AG,Kambs e.G.,Dogan-Ziegert-Allee,U-ST (MUSIKER),BAVARIA,1,3,3,0,2014-04-24,2,163.20
9949751,Geisler Reinhardt AG,Hesse Keudel GbR,Theodora-Kobelt-Platz,U-ST (MUSIKER),THURINGIA,1,3,5,0,2014-04-26,2,106.20
9949924,Ehlert Briemer GmbH & Co. KG,Gotthard,Senol-Bachmann-Straße,U-ST (MUSIKER),RHINELAND-PALATINATE,1,3,6,0,2014-04-20,1,84.96
9950662,Pohl Rogge AG & Co. KG,Roht GmbH,Klaus-D.-Jopich-Gasse,U-ST (MUSIKER),BRANDENBURG,1,4,6,0,2014-05-04,1,69.30


## Get the tariffs specific and not-specific for each class 

In [24]:
class_zero_tariffs = []
class_one_tariffs = []
class_two_tariffs = []

never_class_zero = []
never_class_one = []
never_class_two = []

tarifs_overlap_in_3_classes = []

for each in df['tarif_bez'].unique():
    classes = df[df['tarif_bez'] == each]["veranst_segment"].values.tolist()
    class_count = len(set(classes))
    if class_count == 2:
        not_occuring_class = list(set([0,1,2]).difference(set(classes)))[0]
        if not_occuring_class == 0:
            never_class_zero.append(each)
        elif not_occuring_class == 1:
            never_class_one.append(each)
        else:
            never_class_two.append(each)

    if class_count == 1:
        each_class = classes[0]
        if each_class == 0:
            class_zero_tariffs.append(each)
        elif each_class == 1:
            class_one_tariffs.append(each)
        else:
            class_two_tariffs.append(each)

    if class_count == 3:
        tarifs_overlap_in_3_classes.append(each)

print(never_class_zero)      
print(len(never_class_zero))
print(never_class_one)
print(len(never_class_one))
print(never_class_two)
print(len(never_class_two))

['U-ST (MUSIKER)', 'M-SP II. 1+2 (MUSIKER)']
2
['U-VK I IM FREIEN NACH QM', 'KONZERT CHORVERBAENDE OHNE EINTRITTSGELD (5%)']
2
['VK I 1 VARIETE- UND KABARETTBETRIEBE KONSUMATION (MUSIKER) NEU', 'E-P', 'VARIETE I MINDESTVERGUETUNG (MUSIKER)', 'E (TONTRAEGER)', 'KONZERT CHORVERBAENDE OHNE EINTRITTSGELD (10%)', 'KONZERT CHORVERBAENDE OHNE EINTRITTSGELD (7,5%)', 'BM I. 1. / 2.']
7


In [25]:
print(len(class_zero_tariffs))
print(len(class_one_tariffs))
print(len(class_two_tariffs))

10
0
2


In [26]:
print(len(tarifs_overlap_in_3_classes))
tarifs_overlap_in_3_classes

64


['U-K (MUSIKER)',
 'U-V II. 1 (+II 2)',
 'U-K (MUSIKER) MINDESTVERGUETUNG',
 'U-K I (+ II 2B) - MIT SONSTIGEM GELDWERTEN VORTEIL',
 'U-K II. (MUSIKER) BIS 50 MIN',
 'WR-VR-B 1 I (MUSIKER)',
 'U-K II. (MUSIKER) BIS 35 MIN',
 'U-K II. (MUSIKER) BIS 5 MIN',
 'U-V III. 2',
 'U-K II. (MUSIKER) BIS 15 MIN',
 'U-V III. 1',
 'U-K II. (MUSIKER) BIS 20 MIN',
 'U-V VI. A',
 'U-K II. (MUSIKER) BIS 30 MIN',
 'U-K II. (MUSIKER) BIS 10 MIN',
 'U-V VI. B',
 'U-VK I (+III 2A) - FUER MOEGLICHEN ZEITZUSCHLAG',
 'U-K II. (MUSIKER) BIS 25 MIN',
 'U-V-KS (MUSIKER)',
 'U-K II (MUSIKER) MINDESTVERGUETUNG',
 'U-V VI. B MINDESTVERGUETUNG',
 'U-K II. (+III 2B) (MUSIKER) BIS 15 MIN',
 'U-K II. (MUSIKER) BIS 40 MIN',
 'U-V II. 1 (+II 2A) KARAOKE',
 'U-K II. (MUSIKER) BIS 45 MIN',
 'U-K II. (+III 2B) (MUSIKER) BIS 40 MIN',
 'E (MUSIKER)',
 'VK I 1 VARIETE- UND KABARETTBETRIEBE (MUSIKER)',
 'WR-KS EINZEL (MUSIKER)',
 'U-K III. 2D) (MUSIKER) - VOR GELADENEN GAESTEN',
 'U-K II. (+III 2B) (MUSIKER) BIS 50 MIN',
 'U-V I

In [41]:
filter_tarif_df = df[df['tarif_bez'] == 'U-K I (+ II 2B) - MIT SONSTIGEM GELDWERTEN VORTEIL']

filter_tarif_segment_0_df = filter_tarif_df[filter_tarif_df['veranst_segment'] == 0]
filter_tarif_segment_1_df = filter_tarif_df[filter_tarif_df['veranst_segment'] == 1]
filter_tarif_segment_2_df = filter_tarif_df[filter_tarif_df['veranst_segment'] == 2]


print(len(filter_tarif_segment_0_df["anonymized_venue"].unique()))
print(len(filter_tarif_segment_1_df["anonymized_venue"].unique()))
print(len(filter_tarif_segment_2_df["anonymized_venue"].unique()))

print(len(filter_tarif_segment_0_df["vg_state"].unique()))
print(len(filter_tarif_segment_1_df["vg_state"].unique()))
print(len(filter_tarif_segment_2_df["vg_state"].unique()))



# filter_tarif_segment_1_df[]

43
37
65
10
12
14


In [73]:
# ten_class_zero_tariffs = ['WR-NWSP I.', 'U-VK II 3', 'PV/ST NR.3(1) KURSE (MUSIK)', 
#  'U-VK I (+III 2A) - FUER MOEGLICHEN ZEITZUSCHLAG SCHNELLERFASSUNG', 
#  'U-VK II 2', 'CHORVERBAND KONZERT E-MUSIK OHNE EINTRITTSGELD', 
#  'WR-BEST (MUSIKER)', 'PV/ST NR. 3(1) ZIFF. 3', 
#  'WR-KS I. 2 EINZEL (MUSIKER)', 'WR-HZ (MUSIKER)']

six_class_0_tariffs = ['WR-NWSP I.', 'PV/ST NR.3(1) KURSE (MUSIK)',  
 'U-VK II 2', 'CHORVERBAND KONZERT E-MUSIK OHNE EINTRITTSGELD', 
'PV/ST NR. 3(1) ZIFF. 3', 
 'WR-KS I. 2 EINZEL (MUSIKER)']

two_class_two_tariffs = ['U-ST I (MUSIKER) NL', 'RV/L NR. 1 (3) I 1. PROZENTABRECHNUNG TONTRAEGER']

never_class_zero = ['U-ST (MUSIKER)', 'M-SP II. 1+2 (MUSIKER)']

# never_class_one = ['U-VK I IM FREIEN NACH QM', 'KONZERT CHORVERBAENDE OHNE EINTRITTSGELD (5%)']

never_class_one = ['U-VK I IM FREIEN NACH QM']

never_class_two = ['VK I 1 VARIETE- UND KABARETTBETRIEBE KONSUMATION (MUSIKER) NEU', 
                   'E-P', 'VARIETE I MINDESTVERGUETUNG (MUSIKER)', 'E (TONTRAEGER)', 
                   'KONZERT CHORVERBAENDE OHNE EINTRITTSGELD (10%)', 
                   'KONZERT CHORVERBAENDE OHNE EINTRITTSGELD (7,5%)']

# never_class_two = ['VK I 1 VARIETE- UND KABARETTBETRIEBE KONSUMATION (MUSIKER) NEU', 
#                    'E-P', 'VARIETE I MINDESTVERGUETUNG (MUSIKER)', 'E (TONTRAEGER)', 
#                    'KONZERT CHORVERBAENDE OHNE EINTRITTSGELD (10%)', 
#                    'KONZERT CHORVERBAENDE OHNE EINTRITTSGELD (7,5%)', 
#                    'BM I. 1. / 2.']


# program ='''
# row(t1).

# npp(tabnet_vgsegment(3,T),[0,1,2]) :- row(T). 

# % we assign one probability for each class
# {event(T,C): C=0..2}=1  :- tabnet_vgsegment(prob,+T,-C).

# % It's a mistake if the tarif 56 belongs to class 0 or class 1 
# :- event(T,C), tarif(TA), TA=56, C=0 .
# :- event(T,C), tarif(TA), TA=56, C=1 . 
# '''

# =========================================================================
# 1{vgsegment(0,t1,0); vgsegment(0,t1,1); vgsegment(0,t1,2)}1.

# row(t1).
# event(T,C) :- vgsegment(0,T,C), tarif(TA).


# % It's a mistake if the tarif 56 belongs to class 0 or class 1
# :- event(T,C), tarif(TA), TA=(26;56), C=0 .
# :- event(T,C), tarif(TA), TA=(26;56), C=1 .

# % It's a mistake if the tarif 0..10 belongs to class 1 or class 2.
# :- event(T,C), tarif(TA), TA=(1;77;24;69;23;76), C=1 .
# :- event(T,C), tarif(TA), TA=(1;77;24;69;23;76), C=2 .
   

# % It's a mistake if the tarif 11 or 12 belongs to class 0.
# :- event(T,C), tarif(TA), TA=(55;17), C=0 .

# % It's a mistake if the tarif 13 or 14 belongs to class 1.
# :- event(T,C), tarif(TA), TA=(68), C=1 .

# % It's a mistake if the tarif 15..21 belongs to class 2.
# :- event(T,C), tarif(TA), TA=(73;9;71;4;13;14), C=2 .



# % :- not event(t1,0). 
# % :- not event(1, 0). 
# tarif(21).

In [74]:
from sklearn.preprocessing import LabelEncoder

class_frequency = df1.groupby('veranst_segment')['veranst_segment'].transform('count')
df_sampled = df1.sample(n=300000, weights=class_frequency, random_state=2)

le = LabelEncoder()
df_sampled['tarif_bez'] = le.fit_transform(df_sampled['tarif_bez'])

all_tarifs_le = [e for e in df_sampled['tarif_bez']]

tarif_classes=le.inverse_transform(all_tarifs_le).tolist()

for each_tarif in six_class_0_tariffs:
    index_of_tarif = tarif_classes.index(each_tarif)
    print('The index is: ',index_of_tarif)
    print('The label encoded value is: ',all_tarifs_le[index_of_tarif])

The index is:  6604
The label encoded value is:  77
The index is:  103511
The label encoded value is:  24
The index is:  160357
The label encoded value is:  69
The index is:  1481
The label encoded value is:  1
The index is:  24043
The label encoded value is:  23
The index is:  147785
The label encoded value is:  76


In [46]:
len(df_sampled['tarif_bez'].unique())

80

In [75]:
df2 = pd.read_parquet('/Users/saravana/Documents/Work/Master-Thesis/reclamation-processing/data/export_features_2016_2020_v5.parquet.gzip')

pd.set_option('display.max_columns', None)
display(df2.head())

,venue_10%,venue_15%,venue_20%,venue_25%,venue_30%,venue_35%,venue_40%,venue_45%,venue_5%,venue_50%,venue_55%,venue_60%,venue_65%,venue_70%,venue_75%,venue_80%,venue_85%,venue_90%,venue_95%,venue_count,venue_max,venue_mean,venue_min,venue_std,band_10%,band_15%,band_20%,band_25%,band_30%,band_35%,band_40%,band_45%,band_5%,band_50%,band_55%,band_60%,band_65%,band_70%,band_75%,band_80%,band_85%,band_90%,band_95%,band_count,band_max,band_mean,band_min,band_std,promoter_10%,promoter_15%,promoter_20%,promoter_25%,promoter_30%,promoter_35%,promoter_40%,promoter_45%,promoter_5%,promoter_50%,promoter_55%,promoter_60%,promoter_65%,promoter_70%,promoter_75%,promoter_80%,promoter_85%,promoter_90%,promoter_95%,promoter_count,promoter_max,promoter_mean,promoter_min,promoter_std,vg_inkasso,veranst_segment,vg_state,vg_datum_year,vg_datum_month,vg_datum_day_of_week,vg_datum_season,tarif_bez
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
9938766,24.446,32.78,32.780,32.780,32.78,32.78,32.780,32.78,23.520,35.67,38.56,43.924,47.500,63.635,70.55,70.550,127.687,137.770,206.3885,22.0,210.00,64.186818,23.52,56.986319,210.000,210.0000,210.000,210.0000,210.000,210.00,210.00,210.00,210.0000,210.00,210.00,210.00,210.00,210.00,210.00,210.00,210.00,210.00,210.00,2.0,210.00,210.0000,210.00,0.000000,23.405,32.2325,37.85,37.85,40.775,46.1375,47.60,47.60,20.666,57.625,67.650,67.650,67.875,68.700,79.395,101.934,118.8345,132.513,168.75,12.0,210.00,72.630000,19.28,55.046018,210.00,2,1,0,1,5,0,U-K (MUSIKER)
9939272,85.740,85.74,85.740,85.905,86.07,86.07,86.280,86.42,69.321,86.42,86.42,86.420,86.420,86.420,86.79,87.160,87.160,113.308,130.7400,15.0,130.74,88.551333,31.01,22.254023,47.429,55.6385,63.848,72.0575,80.267,85.74,85.74,85.74,39.2195,85.74,85.74,85.74,85.74,85.74,85.74,85.74,85.74,85.74,85.74,4.0,85.74,72.0575,31.01,27.365000,85.740,85.7400,85.74,85.74,85.740,85.9050,86.07,86.07,58.375,86.070,86.245,86.420,86.420,86.420,86.420,86.420,86.4200,86.420,86.42,11.0,86.42,81.133636,31.01,16.626780,85.06,1,15,0,1,5,0,U-V II. 1 (+II 2)
9939279,91.400,91.50,91.600,91.700,91.80,91.90,92.000,92.10,91.300,92.20,92.20,92.200,92.200,92.200,92.20,92.200,92.200,92.200,92.2000,3.0,92.20,91.866667,91.20,0.577350,0.000,0.0000,0.000,0.0000,0.000,0.00,0.00,0.00,0.0000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.0000,0.00,0.000000,21.800,58.0000,72.96,73.76,73.760,73.7600,73.76,91.50,21.800,93.200,94.200,94.200,174.210,238.928,876.800,1109.100,2882.4420,82243.190,82243.19,75.0,82243.19,12285.786800,21.80,29201.628038,90.20,1,1,0,1,1,0,U-V II. 1 (+II 2)
9939280,22.950,24.10,37.188,45.600,54.72,55.32,59.832,71.30,18.760,118.72,119.68,120.520,129.426,137.840,155.27,207.428,208.300,210.490,230.7200,73.0,379.14,111.181781,16.38,81.382902,184.400,185.4860,193.088,200.6900,206.120,206.12,206.12,206.12,184.4000,206.12,206.12,287.12,428.87,570.62,611.12,611.12,611.12,611.12,611.12,8.0,611.12,352.5650,184.40,214.292661,55.320,55.3200,55.56,87.32,118.720,119.1400,119.32,119.86,54.780,206.120,206.120,206.556,208.300,208.300,208.300,208.300,209.8330,210.490,210.49,23.0,210.49,149.093043,54.72,67.320392,206.12,2,1,0,2,5,1,U-V II. 1 (+II 2)
9939281,22.950,24.10,37.188,45.600,54.72,55.32,59.832,71.30,18.760,118.72,119.68,120.520,129.426,137.840,155.27,207.428,208.300,210.490,230.7200,73.0,379.14,111.181781,16.38,81.382902,184.400,185.4860,193.088,200.6900,206.120,206.12,206.12,206.12,184.4000,206.12,206.12,287.12,428.87,570.62,611.12,611.12,611.12,611.12,611.12,8.0,611.12,352.5650,184.40,214.292661,55.320,55.3200,55.56,87.32,118.720,119.1400,119.32,119.86,54.780,206.120,206.120,206.556,208.300,208.300,208.300,208.300,209.8330,210.490,210.49,23.0,210.49,149.093043,54.72,67.320392,206.12,2,1,0,2,4,1,U-V II. 1 (+II 2)


In [77]:
# copy the data
df_min_max_scaled = df2.copy()

le = LabelEncoder()
df_min_max_scaled['tarif_bez'] = le.fit_transform(df_min_max_scaled['tarif_bez'])

columns = ['venue_10%', 'venue_15%', 'venue_20%', 'venue_25%', 'venue_30%', 
    'venue_35%', 'venue_40%', 'venue_45%', 'venue_5%', 'venue_50%', 
    'venue_55%', 'venue_60%', 'venue_65%', 'venue_70%', 'venue_75%', 
    'venue_80%', 'venue_85%', 'venue_90%', 'venue_95%', 'venue_count', 
    'venue_max', 'venue_mean', 'venue_min', 'venue_std', 
    
    'band_10%', 'band_15%', 'band_20%', 'band_25%', 'band_30%', 
    'band_35%', 'band_40%', 'band_45%', 'band_5%', 'band_50%', 
    'band_55%', 'band_60%', 'band_65%', 'band_70%', 'band_75%', 
    'band_80%', 'band_85%', 'band_90%', 'band_95%', 'band_count', 
    'band_max', 'band_mean', 'band_min', 'band_std', 

    'promoter_10%', 'promoter_15%', 'promoter_20%', 'promoter_25%', 'promoter_30%', 
    'promoter_35%', 'promoter_40%', 'promoter_45%', 'promoter_5%', 'promoter_50%', 
    'promoter_55%', 'promoter_60%', 'promoter_65%', 'promoter_70%', 'promoter_75%', 
    'promoter_80%', 'promoter_85%', 'promoter_90%', 'promoter_95%', 'promoter_count', 
    'promoter_max', 'promoter_mean', 'promoter_min', 'promoter_std',
    
    'vg_state', 'vg_datum_year', 
    'vg_datum_month', 'vg_datum_day_of_week', 'vg_datum_season', 'tarif_bez']

# apply normalization technique
for each_column in columns:
    df_min_max_scaled[each_column] = (df_min_max_scaled[each_column] - df_min_max_scaled[each_column].min()) / (df_min_max_scaled[each_column].max() - df_min_max_scaled[each_column].min())    

# Return normalized data
display(df_min_max_scaled)

,venue_10%,venue_15%,venue_20%,venue_25%,venue_30%,venue_35%,venue_40%,venue_45%,venue_5%,venue_50%,venue_55%,venue_60%,venue_65%,venue_70%,venue_75%,venue_80%,venue_85%,venue_90%,venue_95%,venue_count,venue_max,venue_mean,venue_min,venue_std,band_10%,band_15%,band_20%,band_25%,band_30%,band_35%,band_40%,band_45%,band_5%,band_50%,band_55%,band_60%,band_65%,band_70%,band_75%,band_80%,band_85%,band_90%,band_95%,band_count,band_max,band_mean,band_min,band_std,promoter_10%,promoter_15%,promoter_20%,promoter_25%,promoter_30%,promoter_35%,promoter_40%,promoter_45%,promoter_5%,promoter_50%,promoter_55%,promoter_60%,promoter_65%,promoter_70%,promoter_75%,promoter_80%,promoter_85%,promoter_90%,promoter_95%,promoter_count,promoter_max,promoter_mean,promoter_min,promoter_std,vg_inkasso,veranst_segment,vg_state,vg_datum_year,vg_datum_month,vg_datum_day_of_week,vg_datum_season,tarif_bez
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
9938766,0.000095,0.000075,0.000065,0.000044,0.000042,0.000042,0.000035,0.000033,0.000098,0.000036,0.000039,0.000044,0.000048,0.000064,0.000071,0.000071,0.000128,0.000134,0.000200,0.000404,0.000204,0.000098,1.281676e-04,0.000123,0.000204,0.000204,0.000204,0.000204,0.000204,0.000204,0.000204,0.000204,0.000204,0.000204,0.000204,0.000204,0.000204,0.000204,0.000204,0.000204,0.000204,0.000204,0.000204,0.001059,0.000204,0.000204,0.000204,0.000000,0.000024,0.000032,0.000038,0.000038,0.000041,0.000046,0.000048,0.000048,0.000021,0.000058,0.000068,0.000068,0.000068,0.000069,0.000078,0.000099,0.000115,0.000129,0.000164,0.002113,0.000204,0.000073,1.287874e-04,0.000140,210.00,2,0.066667,0.000000,0.090909,0.833333,0.000000,0.348837
9939272,0.000334,0.000195,0.000170,0.000116,0.000111,0.000111,0.000091,0.000087,0.000288,0.000087,0.000087,0.000087,0.000087,0.000087,0.000087,0.000088,0.000087,0.000110,0.000127,0.000276,0.000127,0.000135,1.689829e-04,0.000048,0.000046,0.000054,0.000062,0.000070,0.000078,0.000083,0.000083,0.000083,0.000038,0.000083,0.000083,0.000083,0.000083,0.000083,0.000083,0.000083,0.000083,0.000083,0.000083,0.002118,0.000083,0.000070,0.000030,0.000038,0.000086,0.000086,0.000086,0.000086,0.000086,0.000086,0.000087,0.000087,0.000059,0.000087,0.000087,0.000087,0.000087,0.000087,0.000085,0.000084,0.000084,0.000084,0.000084,0.001937,0.000084,0.000081,2.071420e-04,0.000042,85.06,1,1.000000,0.000000,0.090909,0.833333,0.000000,0.697674
9939279,0.000356,0.000208,0.000182,0.000123,0.000118,0.000119,0.000097,0.000093,0.000379,0.000093,0.000093,0.000093,0.000093,0.000093,0.000093,0.000093,0.000092,0.000090,0.000090,0.000055,0.000090,0.000140,4.969764e-04,0.000001,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000022,0.000058,0.000073,0.000074,0.000074,0.000074,0.000074,0.000092,0.000022,0.000094,0.000095,0.000095,0.000175,0.000240,0.000859,0.001077,0.002800,0.079881,0.079881,0.013204,0.079881,0.012336,1.456206e-04,0.074369,90.20,1,0.066667,0.000000,0.090909,0.166667,0.000000,0.697674
9939280,0.000090,0.000055,0.000074,0.000061,0.000071,0.000071,0.000063,0.000072,0.000078,0.000119,0.000120,0.000121,0.000130,0.000139,0.000156,0.000209,0.000208,0.000204,0.000224,0.001341,0.000368,0.000170,8.925958e-05,0.000176,0.000179,0.000180,0.000188,0.000195,0.000200,0.000200,0.000200,0.000200,0.000179,0.000200,0.000200,0.000279,0.000417,0.000554,0.000594,0.000594,0.000594,0.000594,0.000594,0.004235,0.000594,0.000342,0.000179,0.000295,0.000056,0.000056,0.000056,0.000088,0.000119,0.000120,0.000120,0.000121,0.000055,0.000207,0.000207,0.000208,0.000210,0.000210,0.000204,0.000202,0.000204,0.000204,0.000204,0.004049,0.000204,0.000150,3.655211e-04,0.000171,206.12,2,0.066667,0.000000,0.181818,0.833333,0.333333,0.697674
9939281,0.000090,0.000055,0.000074,0.000061,0.000071,0.000071,0.000063,0.000072,0.000078,0.000119,0.000120,0.000121,0.000130

In [ ]:
values_u_st = []
for each in tarif_u_st_df['tarif_bez'].values:
    values_u_st.append(each)

In [ ]:
new_tarif_u_st_df = pd.DataFrame({'tarif_bez':values_u_st})

In [ ]:
fig, ax = plt.subplots()
new_tarif_u_st_df['tarif_bez'].value_counts().plot(ax=ax, kind='bar', xlabel='Tarifs', ylabel='Frequency')
plt.show()
# U-ST I. (MUSIKER) - 106539
# U-ST I (MUSIKER) NL - 27919
# U-ST (MUSIKER) - 3928

In [ ]:
tarif_u_st_df.shape

### vg_state

In [ ]:
new_tarif_u_st_df['state'] = tarif_u_st_df['vg_state'].values

In [ ]:
new_tarif_u_st_df['vg_raum_keywords'] = tarif_u_st_df['vg_raum_keywords'].values

In [ ]:
new_tarif_u_st_df['inkasso'] = tarif_u_st_df['vg_inkasso'].values

In [ ]:
new_tarif_u_st_df['veranst_segment'] = tarif_u_st_df['veranst_segment'].values

In [ ]:
new_tarif_u_st_df['venue'] = [','.join(map(str, l)) for l in new_tarif_u_st_df['vg_raum_keywords']]

In [ ]:
u_st_nl_df = df.loc[df['tarif_bez']=='U-ST I (MUSIKER) NL']
u_st_nl_df.shape

In [ ]:
display(new_tarif_u_st_df.head())

In [ ]:
# u_st_df = new_tarif_u_st_df.loc[new_tarif_u_st_df['tarif_bez']=='U-ST (MUSIKER)'] # veranst segment ==> 1 to 2 - {>50€}
u_st_i_df = new_tarif_u_st_df.loc[new_tarif_u_st_df['tarif_bez']=='U-ST I. (MUSIKER)'] # veranst segment ==> 0 to 2 - {>0€}
# u_st_nl_df = new_tarif_u_st_df.loc[new_tarif_u_st_df['tarif_bez']=='U-ST I (MUSIKER) NL'] # veranst segment ==> 2 to 2 {>100€}

state = ['bavaria', 'thuringia', 'baden-wuerttemberg', 'rhineland-palatinate', 
         'brandenburg', 'berlin', 'lower saxony', 'schleswig-holstein', 'hamburg', 
         'hesse', 'saarland', 'north rhine-westphalia', 'saxony-anhalt', 
         'saxony', 'mecklenburg-western pomerania', 'bremen']

for each in state:
    st = each.upper()
    print('--------State---------: ',st)
    specific_state_df = u_st_i_df.loc[u_st_i_df['state']==st]
    # print('Venues available here are: ',specific_state_df.venue.value_counts())
    # print(specific_state_df.venue.value_counts())
    print('Total number of venues: ',len(specific_state_df.venue.values))

    if st == 'SAXONY':
        kulturzentrum_df = specific_state_df.loc[specific_state_df.venue.str.contains('kulturzentrum')==True]
        print('The collection in kulturzentrum is :', kulturzentrum_df.inkasso)
        print('The kulturzentrum min collection is: {}, max is:{}'.format(kulturzentrum_df.inkasso.min(), kulturzentrum_df.inkasso.max()))
        print('The venue is: ', kulturzentrum_df.venue)

        kirche_df = specific_state_df.loc[specific_state_df.venue.str.contains('kirche')==True]
        # print('The collection in kulturzentrum is :', kirche_df.inkasso)
        print('The kirche min collection is: {}, max is:{}'.format(kirche_df.inkasso.min(), kirche_df.inkasso.max()))
        # print('The venue is: ', kirche_df.venue)

        hotel_df = specific_state_df.loc[specific_state_df.venue.str.contains('hotel')==True]
        # print('The collection in hotel is :', hotel_df.inkasso)
        print('The hotel min collection is: {}, max is:{}'.format(hotel_df.inkasso.min(), hotel_df.inkasso.max()))

        cafe_df = specific_state_df.loc[specific_state_df.venue.str.contains('cafe')==True]
        # print('The collection in cafe is :', cafe_df.inkasso)
        print('The cafe min collection is: {}, max is:{}'.format(cafe_df.inkasso.min(), cafe_df.inkasso.max()))

        theater_df = specific_state_df.loc[specific_state_df.venue.str.contains('theater')==True]
        # print('The collection in theater is :', theater_df.inkasso)
        print('The theater min collection is: {}, max is:{}'.format(theater_df.inkasso.min(), theater_df.inkasso.max()))

        club_df = specific_state_df.loc[specific_state_df.venue.str.contains('club')==True]
        # print('The collection in club is :', club_df.inkasso)
        print('The club min collection is: {}, max is:{}'.format(club_df.inkasso.min(), club_df.inkasso.max()))

        halle_df = specific_state_df.loc[specific_state_df.venue.str.contains('halle')==True]
        # print('The collection in halle is :', halle_df.inkasso)
        print('The halle min collection is: {}, max is:{}'.format(halle_df.inkasso.min(), halle_df.inkasso.max()))

        gaststaette_df = specific_state_df.loc[specific_state_df.venue.str.contains('gaststaette')==True]
        # print('The collection in gaststaette is :', gaststaette_df.inkasso)
        print('The gaststaette min collection is: {}, max is:{}'.format(gaststaette_df.inkasso.min(), gaststaette_df.inkasso.max()))

        festhalle_df = specific_state_df.loc[specific_state_df.venue.str.contains('festhalle')==True]
        # print('The collection in festhalle is :', festhalle_df.inkasso)
        print('The festhalle min collection is: {}, max is:{}'.format(festhalle_df.inkasso.min(), festhalle_df.inkasso.max()))

        festzelt_df = specific_state_df.loc[specific_state_df.venue.str.contains('festzelt')==True]
        # print('The collection in festzelt is :', festzelt_df.inkasso)
        print('The festzelt min collection is: {}, max is:{}'.format(festzelt_df.inkasso.min(), festzelt_df.inkasso.max()))

        schloss_df = specific_state_df.loc[specific_state_df.venue.str.contains('schloss')==True]
        # print('The collection in schloss is :', schloss_df.inkasso)
        print('The schloss min collection is: {}, max is:{}'.format(schloss_df.inkasso.min(), schloss_df.inkasso.max()))

        pub_df = specific_state_df.loc[specific_state_df.venue.str.contains('pub')==True]
        # print('The collection in pub is :', pub_df.inkasso)
        print('The pub min collection is: {}, max is:{}'.format(pub_df.inkasso.min(), pub_df.inkasso.max()))

        stadthalle_df = specific_state_df.loc[specific_state_df.venue.str.contains('stadthalle')==True]
        # print('The collection in stadthalle is :', stadthalle_df.inkasso)
        print('The stadthalle min collection is: {}, max is:{}'.format(stadthalle_df.inkasso.min(), stadthalle_df.inkasso.max()))

        park_df = specific_state_df.loc[specific_state_df.venue.str.contains('park')==True]
        # print('The collection in park is :', park_df.inkasso)
        print('The park min collection is: {}, max is:{}'.format(park_df.inkasso.min(), park_df.inkasso.max()))

        gasthof_df = specific_state_df.loc[specific_state_df.venue.str.contains('gasthof')==True]
        # print('The collection in gasthof is :', gasthof_df.inkasso)
        print('The gasthof min collection is: {}, max is:{}'.format(gasthof_df.inkasso.min(), gasthof_df.inkasso.max()))

        kabarett_df = specific_state_df.loc[specific_state_df.venue.str.contains('kabarett')==True]
        # print('The collection in kabarett is :', kabarett_df.inkasso)
        print('The kabarett min collection is: {}, max is:{}'.format(kabarett_df.inkasso.min(), kabarett_df.inkasso.max()))

        arena_df = specific_state_df.loc[specific_state_df.venue.str.contains('arena')==True]
        # print('The collection in arena is :', arena_df.inkasso)
        print('The arena min collection is: {}, max is:{}'.format(arena_df.inkasso.min(), arena_df.inkasso.max()))

        schlachthof_df = specific_state_df.loc[specific_state_df.venue.str.contains('schlachthof')==True]
        # print('The collection in schlachthof is :', schlachthof_df.inkasso)
        print('The schlachthof min collection is: {}, max is:{}'.format(schlachthof_df.inkasso.min(), schlachthof_df.inkasso.max()))

        wandelhalle_df = specific_state_df.loc[specific_state_df.venue.str.contains('wandelhalle')==True]
        # print('The collection in wandelhalle is :', wandelhalle_df.inkasso)
        print('The wandelhalle min collection is: {}, max is:{}'.format(wandelhalle_df.inkasso.min(), wandelhalle_df.inkasso.max()))

        turnhalle_df = specific_state_df.loc[specific_state_df.venue.str.contains('turnhalle')==True]
        # print('The collection in turnhalle is :', turnhalle_df.inkasso)
        print('The turnhalle min collection is: {}, max is:{}'.format(turnhalle_df.inkasso.min(), turnhalle_df.inkasso.max()))

        buergerhaus_df = specific_state_df.loc[specific_state_df.venue.str.contains('buergerhaus')==True]
        # print('The collection in buergerhaus is :', buergerhaus_df.inkasso)
        print('The buergerhaus min collection is: {}, max is:{}'.format(buergerhaus_df.inkasso.min(), buergerhaus_df.inkasso.max()))

        museum_df = specific_state_df.loc[specific_state_df.venue.str.contains('museum')==True]
        # print('The collection in museum is :', museum_df.inkasso)
        print('The museum min collection is: {}, max is:{}'.format(museum_df.inkasso.min(), museum_df.inkasso.max()))

        rathaus_df = specific_state_df.loc[specific_state_df.venue.str.contains('rathaus')==True]
        # print('The collection in rathaus is :', rathaus_df.inkasso)
        print('The rathaus min collection is: {}, max is:{}'.format(rathaus_df.inkasso.min(), rathaus_df.inkasso.max()))

        staatsbad_df = specific_state_df.loc[specific_state_df.venue.str.contains('staatsbad')==True]
        # print('The collection in staatsbad is :', staatsbad_df.inkasso)
        print('The staatsbad min collection is: {}, max is:{}'.format(staatsbad_df.inkasso.min(), staatsbad_df.inkasso.max()))

        zelt_df = specific_state_df.loc[specific_state_df.venue.str.contains('zelt')==True]
        # print('The collection in zelt is :', zelt_df.inkasso)
        print('The zelt min collection is: {}, max is:{}'.format(zelt_df.inkasso.min(), zelt_df.inkasso.max()))

        jazz_df = specific_state_df.loc[specific_state_df.venue.str.contains('jazz')==True]
        # print('The collection in jazz is :', jazz_df.inkasso)
        print('The jazz min collection is: {}, max is:{}'.format(jazz_df.inkasso.min(), jazz_df.inkasso.max()))

        forum_df = specific_state_df.loc[specific_state_df.venue.str.contains('forum')==True]
        # print('The collection in forum is :', forum_df.inkasso)
        print('The forum min collection is: {}, max is:{}'.format(forum_df.inkasso.min(), forum_df.inkasso.max()))

        gymnasium_df = specific_state_df.loc[specific_state_df.venue.str.contains('gymnasium')==True]
        # print('The collection in gymnasium is :', gymnasium_df.inkasso)
        print('The gymnasium min collection is: {}, max is:{}'.format(gymnasium_df.inkasso.min(), gymnasium_df.inkasso.max()))

        schule_df = specific_state_df.loc[specific_state_df.venue.str.contains('schule')==True]
        # print('The collection in schule is :', schule_df.inkasso)
        print('The schule min collection is: {}, max is:{}'.format(schule_df.inkasso.min(), schule_df.inkasso.max()))

        sporthalle_df = specific_state_df.loc[specific_state_df.venue.str.contains('sporthalle')==True]
        # print('The collection in sporthalle is :', sporthalle_df.inkasso)
        print('The sporthalle min collection is: {}, max is:{}'.format(sporthalle_df.inkasso.min(), sporthalle_df.inkasso.max()))

    # # indices = specific_state_df.index[specific_state_df.venue.str.contains('kulturzentrum').any()==True].tolist()
    # print('The venue kulturzentrum is present of following indices: ',specific_state_df.venue.str.contains('kulturzentrum').sum())
    
    # if specific_state_df.venue.str.contains('kulturzentrum').any():
    #     print('Contains kulturzentrum--------------------------------------------------------------')
    # zero_to_fifty_df = specific_state_df.loc[specific_state_df['veranst_segment']==0]
    # print('The venues where inkasso is less than 50€ :',zero_to_fifty_df.vg_raum_keywords)
    # print('The max inkasso: ', specific_state_df.inkasso.max())
    # print('The min inkasso: ', specific_state_df.inkasso.min())
    # print('The max segment: ', specific_state_df.veranst_segment.max())
    # print('The min segment: ', specific_state_df.veranst_segment.min())
    # print('------------------------')

In [ ]:
# VENUES 
# place_kirche	place_hotel	place_cafe	
# place_theater	place_club	place_halle	
# place_gaststaette	place_festhalle	place_kulturzentrum	
# place_festzelt	place_schloss	place_pub	
# place_stadthalle	place_park	place_gasthof	
# place_kabarett	place_arena	place_schlachthof	
# place_wandelhalle	place_turnhalle	place_buergerhaus	
# place_museum	place_rathaus	place_staatsbad	
# place_zelt	place_jazz	place_forum	
# place_gymnasium	place_schule	place_sporthalle

In [ ]:
venues = ['kirche', 'hotel', 'cafe',
'theater', 'club', 'halle',	
'gaststaette', 'festhalle', 'kulturzentrum', 	
'festzelt', 'schloss', 'pub', 	
'stadthalle', 'park', 'gasthof', 
'kabarett', 'arena', 'schlachthof', 
'wandelhalle', 'turnhalle', 'buergerhaus', 
'museum', 'rathaus', 'staatsbad', 
'zelt', 'jazz', 'forum', 
'gymnasium', 'schule', 'sporthalle']

In [ ]:
# tarif_u_st_df.plot(x='tarif_bez', y=['state_bavaria'], kind='line', figsize=(10,10))
chart = sns.relplot(data=tarif_u_st_df, x='tarif_bez', y='vg_state', style='place_kirche', kind='line')

In [ ]:
flat_list_vg_raum_keywords = [item for sublist in tarif_u_st_df['vg_raum_keywords'] for item in sublist]

In [ ]:
tarif_u_st_in_one_state_df = tarif_u_st_df.loc[tarif_u_st_df['vg_state']=='SAXONY']

In [ ]:
display(tarif_u_st_in_one_state_df.head())

In [ ]:
chart = sns.countplot(data=tarif_u_st_df, x='vg_raum_wo_stopwords', hue='vg_state')

### Get all unique values of Tarif that contains 'U-V'

In [ ]:
tarif_u_v_df = df.loc[df['tarif_bez'].str.contains('U-V ')].copy()

In [ ]:
tarif_u_v_df.shape

In [ ]:
uv_values = []
for each in tarif_u_v_df['tarif_bez'].values:
    uv_values.append(each)

In [ ]:
new_tarif_u_v_df = pd.DataFrame({'tarif_bez':uv_values})

In [ ]:
fig, ax = plt.subplots()
new_tarif_u_v_df['tarif_bez'].value_counts().plot(ax=ax, kind='bar', xlabel='Tarifs', ylabel='Frequency')
plt.show()